\begin{equation} 
\label{obj.func_stat}
Z=\min \sum_{t=1}^T S_t Y_t+\sum_{t=1}^T h_t\left(\sum_{\tau=1}^tX_{\tau} -\mathbb{E}\left[AD _t\right]\right)
\end{equation}
\begin{equation}\label{equ2_stat}
X_t\leq Cap_t Y_t,  
  \forall  t=1 \ldots T,
\end{equation}
\begin{equation}
\sum_{\tau=1}^t X_\tau \geq F_{AD_t}^{-1}(1-\alpha)\quad\forall t=1 \ldots T. 
\end{equation}
\begin{equation}\label{equ4_stat}
X_t \geq  0,
  \forall  t=1 \ldots T,
\end{equation}
\begin{equation}\label{equ5_stat}
 Y_t \in\{0,1\},
%\forall \quad  1 \leq t \leq T.
   \forall t=1 \ldots T.
\end{equation}

In [15]:
# In this model, we needed quantile function for the summation of uniform dist. and we approximated through sample generation
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
from scipy.stats import uniform
import numpy as np
import random
import time
model1= AbstractModel()
L1=10 #lower bound for uniform distribution
L2=50 #upper bound for uniform distribution
model1.L1=Param(initialize=L1)
model1.L2=Param(initialize=L2)
T=20 # time periods
model1.T=Param(initialize=T)
model1.t=RangeSet(1,model1.T)
#number of instances to train
I=1000
model1.I=Param(initialize=I)
model1.i=RangeSet(1,model1.I)

eps=0.15 # acceptable violation of chance constraint
######################### Quantile function approximation for Sum uniform dist.#####################
data=[[random.uniform(L1,L2) for i in range(I)] for j in range(T)]
res =[data[0]]
for k in range(1,len(data)):
    tempp=[]
    for j in range(0,I):
                        temp=res[-1][j]+data[k][j]
                        tempp.append(temp)
    res.append(tempp)
    res[-2].sort()
res[-1].sort()  
################################################################################
#test scenatios
model1.Scenario=Param(initialize=3000)
model1.scenario=RangeSet(1,model1.Scenario)

def edc(model1,t):
        return (t* (model1.L1+model1.L2)/2)
def Dt (model1,t,i):
            return random.uniform(model1.L1,model1.L2) 
def deltac (model1,t,i):
        return sum(model1.dti[(t,i)] for t in range(1,t+1))
    
def Dt_test (model1,t,i):
        return random.uniform(model1.L1,model1.L2) 
def deltac_test (model1,t,i):
        return sum(model1.dti_test[(t,i)] for t in range(1,t+1))
model1.c=Param(model1.t, initialize =100) #capacity of production ;assigne a very big number for uncapacitated problem
model1.f=Param(model1.t, initialize=50) # ordering cost
model1.h=Param(model1.t, initialize=1,mutable=True) # inventory holding cost per unit
model1.dti= Param(model1.t,model1.i,initialize=Dt)
model1.EDC=Param(model1.t,initialize=edc)
model1.dti_test=Param(model1.t,model1.scenario,initialize=Dt_test)
model1.deltaC_test=Param(model1.t,model1.scenario,initialize=deltac_test) #Accumulated demand 
model1.x=Var(model1.t, within=NonNegativeReals)
model1.y=Var(model1.t, within=Binary)
model1.satisfaction=Var(model1.t,model1.i,within=Binary)

def cons41 (model1,t):
            return model1.x[t]<=model1.c[t] * model1.y[t]
model1.Cons41a=Constraint(model1.t,rule=cons41)

#######################Individual chance constraint using quantile approximation###########################
def indivChance(model1,t):
        return sum(model1.x[teta] for teta in range(1,t+1))>=res[t-1][-round(eps*I)]
model1.ConsChance=Constraint(model1.t,rule=indivChance)
#############################################################################
def obj_rule(model1):        
             return  sum(model1.f[t]*model1.y[t] for t in model1.t)+sum(model1.h[t] *(model1.x[1]+sum(model1.x[tt] for tt in range(2,t+1)) -model1.EDC[t])for t in model1.t) 
model1.of=Objective(rule=obj_rule,sense=minimize)
opt=SolverFactory('gurobi')
instance1=model1.create_instance()

start_time = time.time()
results1=opt.solve(instance1)
print("--- %s seconds ---" % (time.time() - start_time))

if (results1.solver.status == SolverStatus.ok) and (results1.solver.termination_condition == TerminationCondition.optimal):
        print (" problem is feasible and optimal")
elif results1.solver.termination_condition == TerminationCondition.infeasible:
        print ("problem is infeasible")
else:
       print("something else is wrong")
print(str(results1))
  
print("obj :",value(instance1.of))
######### posterior individual probability calculation###############################
p_ind=[]
for t in instance1.t:
    k=0
    for j in instance1.scenario:
        if  sum(value(instance1.x[teta]) for teta in range(1,t+1))<instance1.deltaC_test[t,j]:
            k=k+1
    p=1-k/instance1.Scenario
    p_ind.append(p)
P_individual=sum(p_ind)/instance1.T
print("mean P_per_period:",P_individual)
print("min",min(p_ind))
################################posterior Joint probability calculation################################
Hold=0
for j in instance1.scenario:#iterating over test scenarios
        hold=0       
        for t in instance1.t:
                if sum(value(instance1.x[teta])for teta in range(1,t+1))>=instance1.deltaC_test[t,j]:
                    hold=hold+1
        if hold==instance1.T:
                        Hold=Hold+1
probability=Hold/instance1.Scenario
print("joint probability:",probability)
    

--- 0.3749516010284424 seconds ---
 problem is feasible and optimal

Problem: 
- Name: x1
  Lower bound: 1597.2909090111198
  Upper bound: 1597.2909090111204
  Number of objectives: 1
  Number of constraints: 40
  Number of variables: 41
  Number of binary variables: 20
  Number of integer variables: 20
  Number of continuous variables: 21
  Number of nonzeros: 250
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.029999971389770508
  Error rc: 0
  Time: 0.3239283561706543
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

obj : 1597.29090901112
mean P_per_period: 0.9090166666666667
min 0.8296666666666667
joint probability: 0.588
